In [70]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [71]:
df = pd.read_excel('Retail_clean.xlsx')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year,month,day,Time,Total Price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010,12,Wednesday,08:26 AM,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,Wednesday,08:26 AM,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010,12,Wednesday,08:26 AM,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,Wednesday,08:26 AM,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,Wednesday,08:26 AM,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397879,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,2011,12,Friday,12:50 PM,10.20
397880,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,2011,12,Friday,12:50 PM,12.60
397881,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,2011,12,Friday,12:50 PM,16.60
397882,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,2011,12,Friday,12:50 PM,16.60


In [72]:
df.describe()

,InvoiceNo,Quantity,InvoiceDate,UnitPrice,CustomerID,year,month,Total Price
count,397884.000000,397884.000000,397884,397884.000000,397884.000000,397884.000000,397884.000000,397884.000000
mean,560616.934451,12.988238,2011-07-10 23:41:23.511023360,3.116488,15294.423453,2010.934260,7.612475,22.397000
min,536365.000000,1.000000,2010-12-01 08:26:00,0.001000,12346.000000,2010.000000,1.000000,0.001000
25%,549234.000000,2.000000,2011-04-07 11:12:00,1.250000,13969.000000,2011.000000,5.000000,4.680000
50%,561893.000000,6.000000,2011-07-31 14:39:00,1.950000,15159.000000,2011.000000,8.000000,11.800000
75%,572090.000000,12.000000,2011-10-20 14:33:00,3.750000,16795.000000,2011.000000,11.000000,19.800000
max,581587.000000,80995.000000,2011-12-09 12:50:00,8142.750000,18287.000000,2011.000000,12.000000,168469.600000
std,13106.117773,179.331775,NaN,22.097877,1713.141560,0.247828,3.416520,309.071041


In [73]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
year           0
month          0
day            0
Time           0
Total Price    0
dtype: int64

In [74]:
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (df['InvoiceDate'].max() - x.max()).days,
    'InvoiceNo': 'nunique',
    'Total Price': 'sum'
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

In [75]:
rfm

,CustomerID,Recency,Frequency,Monetary
0,12346,325,1,77183.60
1,12347,1,7,4310.00
2,12348,74,4,1797.24
3,12349,18,1,1757.55
4,12350,309,1,334.40
...,...,...,...,...
4333,18280,277,1,180.60
4334,18281,180,1,80.82
4335,18282,7,2,178.05
4336,18283,3,16,2094.88


In [76]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1], duplicates='drop')
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5], duplicates='drop')
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5], duplicates='drop')

rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

def rfm_segment(row):
    r = row['R_Score']
    f = row['F_Score']
    m = row['M_Score']
    
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'            
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Loyal Customers'     
    elif r >= 4 and f >= 2:
        return 'Potential Loyalists'   
    elif r >= 3 and f >= 1:
        return 'At Risk'              
    elif r <= 2 and f >= 1:
        return 'Can\'t Lose Them'      
    elif r <= 2 and f <= 2:
        return 'Lost'                
    elif r >= 3 and f <= 2:
        return 'Promising'            
    else:
        return 'New Customers'         

rfm['Segment'] = rfm.apply(rfm_segment, axis=1)

In [77]:
rfm['Segment'].value_counts()

Segment
Can't Lose Them        1708
Champions               962
Loyal Customers         758
At Risk                 578
Potential Loyalists     332
Name: count, dtype: int64

In [101]:
rfm[['CustomerID', 'Recency', 'Frequency', 'Monetary', 'RFM_Score', 'Segment']].reset_index()
rfm

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
0,12346,325,1,77183.60,1,1,5,115,Can't Lose Them
1,12347,1,7,4310.00,5,5,5,555,Champions
2,12348,74,4,1797.24,2,4,4,244,Can't Lose Them
3,12349,18,1,1757.55,4,1,4,414,At Risk
4,12350,309,1,334.40,1,1,2,112,Can't Lose Them
...,...,...,...,...,...,...,...,...,...
4333,18280,277,1,180.60,1,2,1,121,Can't Lose Them
4334,18281,180,1,80.82,1,2,1,121,Can't Lose Them
4335,18282,7,2,178.05,5,3,1,531,Potential Loyalists
4336,18283,3,16,2094.88,5,5,5,555,Champions


In [106]:
rfm.to_excel('rfm.xlsx', index=False, sheet_name='RFM_Table')

In [79]:
print(rfm.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'CustomerID': 'count'
}).reset_index().round(1))

               Segment  Recency  Frequency  Monetary  CustomerID
0              At Risk     44.0        1.3     417.7         578
1      Can't Lose Them    192.4        2.0     772.8        1708
2            Champions     11.9       11.1    6038.8         962
3      Loyal Customers     34.7        4.1    1842.6         758
4  Potential Loyalists     16.1        1.9     433.8         332


In [80]:
last_purches = df.groupby('CustomerID')['InvoiceDate'].max()
last_purches

CustomerID
12346   2011-01-18 10:01:00
12347   2011-12-07 15:52:00
12348   2011-09-25 13:13:00
12349   2011-11-21 09:51:00
12350   2011-02-02 16:01:00
                ...        
18280   2011-03-07 09:52:00
18281   2011-06-12 10:53:00
18282   2011-12-02 11:43:00
18283   2011-12-06 12:02:00
18287   2011-10-28 09:29:00
Name: InvoiceDate, Length: 4338, dtype: datetime64[ns]

In [81]:
churned = last_purches < df['InvoiceDate'].max() - pd.DateOffset(months=6)
churned

CustomerID
12346     True
12347    False
12348    False
12349    False
12350     True
         ...  
18280     True
18281    False
18282    False
18283    False
18287    False
Name: InvoiceDate, Length: 4338, dtype: bool

In [96]:
churned.sum()

850

In [82]:
churned_df = churned.reset_index()
churned_df.columns = ['CustomerID','Churned']
churned_df

,CustomerID,Churned
0,12346,True
1,12347,False
2,12348,False
3,12349,False
4,12350,True
...,...,...
4333,18280,True
4334,18281,False
4335,18282,False
4336,18283,False


In [83]:
churned_df['Churned'].value_counts()

Churned
False    3488
True      850
Name: count, dtype: int64

In [98]:
churn_rate = (churned.sum() / (churned.sum() + 3488))*100
churn_rate

19.59428307976026

In [105]:
churned_df.to_excel('churned_df.xlsx', index=False, sheet_name='Churned_Table')

In [104]:
df['CohortMonth'] = df.groupby('CustomerID')['InvoiceDate'].transform('min').dt.to_period('M')
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year,month,day,Time,Total Price,CohortMonth,InvoiceMonth,CohortIndex
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010,12,Wednesday,08:26 AM,15.30,2010-12,2010-12,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,Wednesday,08:26 AM,20.34,2010-12,2010-12,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010,12,Wednesday,08:26 AM,22.00,2010-12,2010-12,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,Wednesday,08:26 AM,20.34,2010-12,2010-12,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,Wednesday,08:26 AM,20.34,2010-12,2010-12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397879,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,2011,12,Friday,12:50 PM,10.20,2011-08,2011-12,4
397880,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,2011,12,Friday,12:50 PM,12.60,2011-08,2011-12,4
397881,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,2011,12,Friday,12:50 PM,16.60,2011-08,2011-12,4
397882,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,2011,12,Friday,12:50 PM,16.60,2011-08,2011-12,4


In [85]:
df['CohortIndex'] = (df['InvoiceMonth'] - df['CohortMonth']).apply(lambda x: x.n) #only month number how many months gone from first purche

cohort_data = df.groupby(['CohortMonth', 'CohortIndex'])['CustomerID'].nunique().reset_index()
cohort_data

,CohortMonth,CohortIndex,CustomerID
0,2010-12,0,885
1,2010-12,1,324
2,2010-12,2,286
3,2010-12,3,340
4,2010-12,4,321
...,...,...,...
86,2011-10,1,86
87,2011-10,2,41
88,2011-11,0,323
89,2011-11,1,36


In [86]:
cohort_table = cohort_data.pivot(index='CohortMonth', 
                                columns='CohortIndex', 
                                values='CustomerID')

cohort_table

CohortIndex,0,1,2,3,4,5,6,7,8,9,10,11,12
CohortMonth,,,,,,,,,,,,,
2010-12,885.0,324.0,286.0,340.0,321.0,352.0,321.0,309.0,313.0,350.0,331.0,445.0,235.0
2011-01,417.0,92.0,111.0,96.0,134.0,120.0,103.0,101.0,125.0,136.0,152.0,49.0,NaN
2011-02,380.0,71.0,71.0,108.0,103.0,94.0,96.0,106.0,94.0,116.0,26.0,NaN,NaN
2011-03,452.0,68.0,114.0,90.0,101.0,76.0,121.0,104.0,126.0,39.0,NaN,NaN,NaN
2011-04,300.0,64.0,61.0,63.0,59.0,68.0,65.0,78.0,22.0,NaN,NaN,NaN,NaN
2011-05,284.0,54.0,49.0,49.0,59.0,66.0,75.0,27.0,NaN,NaN,NaN,NaN,NaN
2011-06,242.0,42.0,38.0,64.0,56.0,81.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,188.0,34.0,39.0,42.0,51.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,169.0,35.0,42.0,41.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
cohort_sizes = cohort_table.iloc[:, 0]
retention_client_at_month = cohort_table.divide(cohort_sizes, axis=0).round(3) * 100

In [88]:
retention_client_at_month

CohortIndex,0,1,2,3,4,5,6,7,8,9,10,11,12
CohortMonth,,,,,,,,,,,,,
2010-12,100.0,36.6,32.3,38.4,36.3,39.8,36.3,34.9,35.4,39.5,37.4,50.3,26.6
2011-01,100.0,22.1,26.6,23.0,32.1,28.8,24.7,24.2,30.0,32.6,36.5,11.8,NaN
2011-02,100.0,18.7,18.7,28.4,27.1,24.7,25.3,27.9,24.7,30.5,6.8,NaN,NaN
2011-03,100.0,15.0,25.2,19.9,22.3,16.8,26.8,23.0,27.9,8.6,NaN,NaN,NaN
2011-04,100.0,21.3,20.3,21.0,19.7,22.7,21.7,26.0,7.3,NaN,NaN,NaN,NaN
2011-05,100.0,19.0,17.3,17.3,20.8,23.2,26.4,9.5,NaN,NaN,NaN,NaN,NaN
2011-06,100.0,17.4,15.7,26.4,23.1,33.5,9.5,NaN,NaN,NaN,NaN,NaN,NaN
2011-07,100.0,18.1,20.7,22.3,27.1,11.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-08,100.0,20.7,24.9,24.3,12.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
#CLV
clv = df.groupby('CustomerID')['Total Price'].sum().reset_index()
clv

,CustomerID,Total Price
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4333,18280,180.60
4334,18281,80.82
4335,18282,178.05
4336,18283,2094.88


In [90]:
customer_span = df.groupby('CustomerID')['InvoiceDate'].agg(['min', 'max'])
customer_span['Span_Years'] = (customer_span['max'] - customer_span['min']).dt.days / 365
customer_span

,min,max,Span_Years
CustomerID,,,
12346,2011-01-18 10:01:00,2011-01-18 10:01:00,0.000000
12347,2010-12-07 14:57:00,2011-12-07 15:52:00,1.000000
12348,2010-12-16 19:09:00,2011-09-25 13:13:00,0.772603
12349,2011-11-21 09:51:00,2011-11-21 09:51:00,0.000000
12350,2011-02-02 16:01:00,2011-02-02 16:01:00,0.000000
...,...,...,...
18280,2011-03-07 09:52:00,2011-03-07 09:52:00,0.000000
18281,2011-06-12 10:53:00,2011-06-12 10:53:00,0.000000
18282,2011-08-05 13:35:00,2011-12-02 11:43:00,0.323288


In [91]:
clv_advanced = df.groupby('CustomerID').agg({
    'Total Price': 'sum'
}).reset_index()

clv_advanced = clv_advanced.merge(customer_span[['Span_Years']], on='CustomerID')
clv_advanced['Annual_CLV'] = clv_advanced['Total Price'] / clv_advanced['Span_Years']

clv_advanced.sort_values('Total Price', ascending=False).head(10)

,CustomerID,Total Price,Span_Years,Annual_CLV
1689,14646,280206.02,0.967123,2.897314e+05
4201,18102,259657.30,1.002740,2.589479e+05
3728,17450,194550.79,0.983562,1.978023e+05
3008,16446,168472.50,0.558904,3.014336e+05
1879,14911,143825.06,1.019178,1.411187e+05
55,12415,124914.53,0.857534,1.456671e+05
1333,14156,117379.63,0.989041,1.186802e+05
3771,17511,91062.38,1.013699,8.983181e+04
2702,16029,81024.84,0.917808,8.828080e+04
0,12346,77183.60,0.000000,inf


In [92]:
clv_rfm = df.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',           # Frequency
    'Total Price': ['sum', 'mean'],   # Total CLV + AOV
    'InvoiceDate': 'count'            # عدد العمليات
}).reset_index()

clv_rfm.columns = ['CustomerID', 'Frequency', 'Total_CLV', 'AOV', 'Num_Purchases']

In [93]:
clv_rfm

,CustomerID,Frequency,Total_CLV,AOV,Num_Purchases
0,12346,1,77183.60,77183.600000,1
1,12347,7,4310.00,23.681319,182
2,12348,4,1797.24,57.975484,31
3,12349,1,1757.55,24.076027,73
4,12350,1,334.40,19.670588,17
...,...,...,...,...,...
4333,18280,1,180.60,18.060000,10
4334,18281,1,80.82,11.545714,7
4335,18282,2,178.05,14.837500,12
4336,18283,16,2094.88,2.771005,756


In [99]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year,month,day,Time,Total Price,CohortMonth,InvoiceMonth,CohortIndex
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010,12,Wednesday,08:26 AM,15.30,2010-12,2010-12,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,Wednesday,08:26 AM,20.34,2010-12,2010-12,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010,12,Wednesday,08:26 AM,22.00,2010-12,2010-12,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,Wednesday,08:26 AM,20.34,2010-12,2010-12,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,Wednesday,08:26 AM,20.34,2010-12,2010-12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397879,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,2011,12,Friday,12:50 PM,10.20,2011-08,2011-12,4
397880,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,2011,12,Friday,12:50 PM,12.60,2011-08,2011-12,4
397881,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,2011,12,Friday,12:50 PM,16.60,2011-08,2011-12,4
397882,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,2011,12,Friday,12:50 PM,16.60,2011-08,2011-12,4


In [ ]:
df['Churned'] = (